In [ ]:
# | default_exp _testing.local_broker

In [ ]:
# | export

import asyncio
import os
import shutil
import socket
import subprocess # nosec - Issue: [B404:blacklist] Consider possible security implications associated with the subprocess module.
import tarfile
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import *

import asyncer
import nest_asyncio
import posix_ipc
import requests
from fastcore.basics import patch
from fastcore.meta import delegates

from fastkafka._components._subprocess import terminate_asyncio_process
from fastkafka._components.helpers import filter_using_signature
from fastkafka._components.logger import get_logger, supress_timestamps
from fastkafka.helpers import in_notebook

In [ ]:
import pytest

from fastkafka.helpers import consumes_messages, produce_messages
from fastkafka.testing import run_script_and_cancel

In [ ]:
# | export

if in_notebook():
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


### Local Kafka

#### Kafka and zookeeper config helpers

In [ ]:
# | export


def get_zookeeper_config_string(
    data_dir: Union[str, Path],  # the directory where the snapshot is stored.
    zookeeper_port: int = 2181,  # the port at which the clients will connect
) -> str:
    """Generates a zookeeeper configuration string that can be exported to file
    and used to start a zookeeper instance.

    Args:
        data_dir: Path to the directory where the zookeepeer instance will save data
        zookeeper_port: Port for clients (Kafka brokes) to connect
    Returns:
        Zookeeper configuration string.

    """

    zookeeper_config = f"""dataDir={data_dir}/zookeeper
clientPort={zookeeper_port}
maxClientCnxns=0
admin.enableServer=false
"""

    return zookeeper_config

In [ ]:
assert (
    get_zookeeper_config_string(data_dir="..")
    == """dataDir=../zookeeper
clientPort=2181
maxClientCnxns=0
admin.enableServer=false
"""
)

assert (
    get_zookeeper_config_string(data_dir="..", zookeeper_port=100)
    == """dataDir=../zookeeper
clientPort=100
maxClientCnxns=0
admin.enableServer=false
"""
)

In [ ]:
# | export


def get_kafka_config_string(
    data_dir: Union[str, Path], zookeeper_port: int = 2181, listener_port: int = 9092
) -> str:
    """Generates a kafka broker configuration string that can be exported to file
    and used to start a kafka broker instance.

    Args:
        data_dir: Path to the directory where the kafka broker instance will save data
        zookeeper_port: Port on which the zookeeper instance is running
        listener_port: Port on which the clients (producers and consumers) can connect
    Returns:
        Kafka broker configuration string.

    """

    kafka_config = f"""broker.id=0

############################# Socket Server Settings #############################

# The address the socket server listens on. If not configured, the host name will be equal to the value of
# java.net.InetAddress.getCanonicalHostName(), with PLAINTEXT listener name, and port 9092.
#   FORMAT:
#     listeners = listener_name://host_name:port
#   EXAMPLE:
#     listeners = PLAINTEXT://your.host.name:9092
listeners=PLAINTEXT://:{listener_port}

# Listener name, hostname and port the broker will advertise to clients.
# If not set, it uses the value for "listeners".
#advertised.listeners=PLAINTEXT://your.host.name:9092

# Maps listener names to security protocols, the default is for them to be the same. See the config documentation for more details
#listener.security.protocol.map=PLAINTEXT:PLAINTEXT,SSL:SSL,SASL_PLAINTEXT:SASL_PLAINTEXT,SASL_SSL:SASL_SSL

# The number of threads that the server uses for receiving requests from the network and sending responses to the network
num.network.threads=3

# The number of threads that the server uses for processing requests, which may include disk I/O
num.io.threads=8

# The send buffer (SO_SNDBUF) used by the socket server
socket.send.buffer.bytes=102400

# The receive buffer (SO_RCVBUF) used by the socket server
socket.receive.buffer.bytes=102400

# The maximum size of a request that the socket server will accept (protection against OOM)
socket.request.max.bytes=104857600


############################# Log Basics #############################

# A comma separated list of directories under which to store log files
log.dirs={data_dir}/kafka_logs

# The default number of log partitions per topic. More partitions allow greater
# parallelism for consumption, but this will also result in more files across
# the brokers.
num.partitions=1

# The number of threads per data directory to be used for log recovery at startup and flushing at shutdown.
# This value is recommended to be increased for installations with data dirs located in RAID array.
num.recovery.threads.per.data.dir=1

offsets.topic.replication.factor=1
transaction.state.log.replication.factor=1
transaction.state.log.min.isr=1

# The number of messages to accept before forcing a flush of data to disk
log.flush.interval.messages=10000

# The maximum amount of time a message can sit in a log before we force a flush
log.flush.interval.ms=1000

# The minimum age of a log file to be eligible for deletion due to age
log.retention.hours=168

# A size-based retention policy for logs. Segments are pruned from the log unless the remaining
# segments drop below log.retention.bytes. Functions independently of log.retention.hours.
log.retention.bytes=1073741824

# The maximum size of a log segment file. When this size is reached a new log segment will be created.
log.segment.bytes=1073741824

# The interval at which log segments are checked to see if they can be deleted according to the retention policies
log.retention.check.interval.ms=300000

# Zookeeper connection string (see zookeeper docs for details).
zookeeper.connect=localhost:{zookeeper_port}

# Timeout in ms for connecting to zookeeper
zookeeper.connection.timeout.ms=18000

# The following configuration specifies the time, in milliseconds, that the GroupCoordinator will delay the initial consumer rebalance.
group.initial.rebalance.delay.ms=0
"""

    return kafka_config

In [ ]:
actual = get_kafka_config_string(data_dir="..", listener_port=9999)
assert "log.dirs=../kafka_logs" in actual
assert "listeners=PLAINTEXT://:9999" in actual

In [ ]:
# | export


class LocalKafkaBroker:
    """LocalKafkaBroker class, used for running unique kafka brokers in tests to prevent topic clashing.

    Attributes:
        lock (ilock.Lock): Lock used for synchronizing the install process between multiple kafka brokers.
    """

    lock = posix_ipc.Semaphore(
        "install_lock:LocalKafkaBroker", posix_ipc.O_CREAT, initial_value=1
    )

    @staticmethod
    def clear_install_semaphore() -> None:
        """Clears semaphore used for synchronizing installation of requirements

        Use this function only if the semaphore is being locked due to crashing process (rarely)
        """
        LocalKafkaBroker.lock.unlink()
        LocalKafkaBroker.lock = posix_ipc.Semaphore(
            "install_lock:LocalKafkaBroker", posix_ipc.O_CREAT, initial_value=1
        )

    @delegates(get_kafka_config_string)  # type: ignore
    @delegates(get_zookeeper_config_string, keep=True)  # type: ignore
    def __init__(
        self,
        topics: Iterable[str] = [],
        *,
        retries: int = 2,
        apply_nest_asyncio: bool = False,
        **kwargs: Dict[str, Any],
    ):
        """Initialises the LocalKafkaBroker object

        Args:
            data_dir: Path to the directory where the zookeepeer instance will save data
            zookeeper_port: Port for clients (Kafka brokes) to connect
            listener_port: Port on which the clients (producers and consumers) can connect
            topics: List of topics to create after sucessfull Kafka broker startup
            retries: Number of retries to create kafka and zookeeper services using random
            apply_nest_asyncio: set to True if running in notebook
            port allocation if the requested port was taken
        """
        self.zookeeper_kwargs = filter_using_signature(
            get_zookeeper_config_string, **kwargs
        )
        self.retries = retries
        self.apply_nest_asyncio = apply_nest_asyncio
        self.kafka_kwargs = filter_using_signature(get_kafka_config_string, **kwargs)
        self.temporary_directory: Optional[TemporaryDirectory] = None
        self.temporary_directory_path: Optional[Path] = None
        self.kafka_task: Optional[asyncio.subprocess.Process] = None
        self.zookeeper_task: Optional[asyncio.subprocess.Process] = None
        self._is_started = False
        self.topics: Iterable[str] = topics
            
    @property
    def is_started(self) -> bool:
        return self._is_started

    @classmethod
    def _install(cls) -> None:
        """Prepares the environment for running Kafka brokers.
        Returns:
           None
        """
        raise NotImplementedError

    async def _start(self) -> str:
        """Starts a local kafka broker and zookeeper instance asynchronously
        Returns:
           Kafka broker bootstrap server address in string format: add:port
        """
        raise NotImplementedError

    def start(self) -> str:
        """Starts a local kafka broker and zookeeper instance synchronously
        Returns:
           Kafka broker bootstrap server address in string format: add:port
        """
        raise NotImplementedError

    def stop(self) -> None:
        """Stops a local kafka broker and zookeeper instance synchronously
        Returns:
           None
        """
        raise NotImplementedError

    async def _stop(self) -> None:
        """Stops a local kafka broker and zookeeper instance synchronously
        Returns:
           None
        """
        raise NotImplementedError

    def get_service_config_string(self, service: str, *, data_dir: Path) -> str:
        """Generates a configuration for a service
        Args:
            data_dir: Path to the directory where the zookeepeer instance will save data
            service: "kafka" or "zookeeper", defines which service to get config string for
        """
        raise NotImplementedError

    async def _start_service(self, service: str = "kafka") -> None:
        """Starts the service according to defined service var
        Args:
            service: "kafka" or "zookeeper", defines which service to start
        """
        raise NotImplementedError

    async def _start_zookeeper(self) -> None:
        """Start a local zookeeper instance
        Returns:
           None
        """
        raise NotImplementedError

    async def _start_kafka(self) -> None:
        """Start a local kafka broker
        Returns:
           None
        """
        raise NotImplementedError

    async def _create_topics(self) -> None:
        """Create missing topics in local Kafka broker
        Returns:
           None
        """
        raise NotImplementedError

    def __enter__(self) -> str:
        #         LocalKafkaBroker._install()
        return self.start()

    def __exit__(self, *args: Any, **kwargs: Any) -> None:
        self.stop()

    async def __aenter__(self) -> str:
        #         LocalKafkaBroker._install()
        return await self._start()

    async def __aexit__(self, *args: Any, **kwargs: Any) -> None:
        await self._stop()

In [ ]:
# print(combine_params(combine_params(LocalKafkaBroker, get_kafka_config_string), get_zookeeper_config_string).__doc__)

In [ ]:
# | notest

! nbdev_export

In [ ]:
script = """
import os
import time
from datetime import datetime
from fastcore.foundation import patch
import posix_ipc

from fastkafka._components.logger import get_logger
from fastkafka.testing import LocalKafkaBroker

pid = os.getpid()

logger = get_logger(f"[PID={pid}]")

@patch(cls_method=True) # type: ignore
def check_cls_lock(cls: LocalKafkaBroker) -> None:
    with cls.lock:
       logger.info(f"Entering: {time.time()}")
       logger.info(f" - {datetime.now()}")
       time.sleep(1)
       logger.info(f"Exiting: {time.time()}")
       logger.info(f" - {datetime.now()}")

broker = LocalKafkaBroker()
broker.check_cls_lock()

"""


def get_times(stdout: str) -> Tuple[float, float]:
    stdout_lines = stdout.split("\n")
    enter_time = float(
        [line.split(" ")[-1] for line in stdout_lines if "Entering" in line][0]
    )
    exit_time = float(
        [line.split(" ")[-1] for line in stdout_lines if "Exiting" in line][0]
    )
    return (enter_time, exit_time)


def check_overlap(intervals: List[Tuple[float]]) -> bool:
    for i, (test_start, test_stop) in enumerate(intervals):
        for start, stop in intervals[i + 1 :]:
            if test_start < start < test_stop or start < test_start < stop:
                return True
    return False


async with asyncer.create_task_group() as tg:
    tx = [tg.soonify(run_script_and_cancel)(script, cancel_after=30) for _ in range(3)]
retvals, stdouts = zip(*[t.value for t in tx])
for retval, stdout in zip(retvals, stdouts):
    print("*" * 100)
    print(f"*   {retval=}")
    print()
    print(stdout.decode("utf-8"))
    print()

times = [get_times(stdout.decode("utf-8")) for stdout in stdouts]
assert not check_overlap(times)

In [ ]:
# | export


def install_java() -> None:
    """Checks if jdk-11 is installed on the machine and installs it if not
    Returns:
       None
    """
    potential_jdk_path = list(Path(os.environ["HOME"] + "/.jdk").glob("jdk-11*"))
    if potential_jdk_path != []:
        logger.info("Java is already installed.")
        if not shutil.which("java"):
            logger.info("But not exported to PATH, exporting...")
            os.environ["PATH"] = os.environ["PATH"] + f":{potential_jdk_path[0]}/bin"
    else:
        logger.info("Installing Java...")
        logger.info(" - installing install-jdk...")
        subprocess.run(["pip", "install", "install-jdk"], check=True)  # nosec
        import jdk

        logger.info(" - installing jdk...")
        jdk_bin_path = jdk.install("11")
        print(jdk_bin_path)
        os.environ["PATH"] = os.environ["PATH"] + f":{jdk_bin_path}/bin"
        logger.info("Java installed.")

In [ ]:
# | notest

install_java()
assert shutil.which("java")
install_java()
assert shutil.which("java")

[INFO] __main__: Java is already installed.
[INFO] __main__: But not exported to PATH, exporting...
[INFO] __main__: Java is already installed.


In [ ]:
# | export


def install_kafka() -> None:
    """Checks if kafka is installed on the machine and installs it if not
    Returns:
       None
    """
    kafka_version = "3.3.2"
    kafka_fname = f"kafka_2.13-{kafka_version}"
    kafka_url = f"https://dlcdn.apache.org/kafka/{kafka_version}/{kafka_fname}.tgz"
    local_path = Path(os.environ["HOME"]) / ".local"
    local_path.mkdir(exist_ok=True, parents=True)
    tgz_path = local_path / f"{kafka_fname}.tgz"
    kafka_path = local_path / f"{kafka_fname}"

    if (kafka_path / "bin").exists():
        logger.info("Kafka is already installed.")
        if not shutil.which("kafka-server-start.sh"):
            logger.info("But not exported to PATH, exporting...")
            os.environ["PATH"] = os.environ["PATH"] + f":{kafka_path}/bin"
    else:
        logger.info("Installing Kafka...")

        response = requests.get(
            kafka_url,
            stream=True,
        )
        try:
            total = response.raw.length_remaining // 128
        except Exception:
            total = None

        with open(tgz_path, "wb") as f:
            for data in tqdm(response.iter_content(chunk_size=128), total=total):
                f.write(data)

        with tarfile.open(tgz_path) as tar:
            for tarinfo in tar:
                tar.extract(tarinfo, local_path)

        os.environ["PATH"] = os.environ["PATH"] + f":{kafka_path}/bin"
        logger.info(f"Kafka installed in {kafka_path}.")

In [ ]:
# | notest

install_kafka()
assert shutil.which("kafka-server-start.sh")
install_kafka()
assert shutil.which("kafka-server-start.sh")

[INFO] __main__: Kafka is already installed.
[INFO] __main__: But not exported to PATH, exporting...
[INFO] __main__: Kafka is already installed.


In [ ]:
# | export


@patch(cls_method=True)  # type: ignore
def _install(cls: LocalKafkaBroker) -> None:
    with cls.lock:
        install_java()
        install_kafka()

In [ ]:
broker = LocalKafkaBroker()
broker._install()
assert shutil.which("java")
assert shutil.which("kafka-server-start.sh")

[INFO] __main__: Java is already installed.
[INFO] __main__: Kafka is already installed.


In [ ]:
# | export
def get_free_port() -> str:
    s = socket.socket()
    s.bind(("127.0.0.1", 0))
    port = str(s.getsockname()[1])
    s.close()
    return port


async def write_config_and_run(
    config: str, config_path: Union[str, Path], run_cmd: str
) -> asyncio.subprocess.Process:
    with open(config_path, "w") as f:
        f.write(config)

    return await asyncio.create_subprocess_exec(
        run_cmd,
        config_path,
        stdout=asyncio.subprocess.PIPE,
        stdin=asyncio.subprocess.PIPE,
    )


@patch  # type: ignore
def get_service_config_string(
    self: LocalKafkaBroker, service: str, *, data_dir: Path
) -> str:
    service_kwargs = getattr(self, f"{service}_kwargs")
    if service == "kafka":
        return get_kafka_config_string(data_dir=data_dir, **service_kwargs)
    else:
        return get_zookeeper_config_string(data_dir=data_dir, **service_kwargs)


@patch  # type: ignore
async def _start_service(self: LocalKafkaBroker, service: str = "kafka") -> None:
    logger.info(f"Starting {service}...")

    if self.temporary_directory_path is None:
        raise ValueError(
            "LocalKafkaBroker._start_service(): self.temporary_directory_path is None, did you initialise it?"
        )

    for i in range(self.retries + 1):
        service_config_path = self.temporary_directory_path / f"{service}.properties"
        service_task = await write_config_and_run(
            self.get_service_config_string(
                service, data_dir=self.temporary_directory_path
            ),
            service_config_path,
            f"{service}-server-start.sh",
        )
        setattr(self, f"{service}_task", service_task)

        logger.info(f"{service} started, sleeping for 5 seconds...")
        await asyncio.sleep(5)

        if service_task.returncode is None:
            break
        elif i < self.retries:
            logger.info(
                f"{service} startup falied, generating a new port and retrying..."
            )
            port = get_free_port()

            portname = service if service != "kafka" else "listener"
            for d in [self.zookeeper_kwargs, self.kafka_kwargs]:
                if f"{portname}_port" in d:
                    d[f"{portname}_port"] = port
            logger.info(f"port={port}")

    if service_task.returncode is not None:
        raise ValueError(
            f"Could not start {service} with params: {getattr(self, f'{service}_kwargs')}"
        )


@patch  # type: ignore
async def _start_kafka(self: LocalKafkaBroker) -> None:
    return await self._start_service("kafka")


@patch  # type: ignore
async def _start_zookeeper(self: LocalKafkaBroker) -> None:
    return await self._start_service("zookeeper")


@patch  # type: ignore
async def _create_topics(self: LocalKafkaBroker) -> None:
    listener_port = self.kafka_kwargs.get("listener_port", 9092)
    bootstrap_server = f"127.0.0.1:{listener_port}"

    async with asyncer.create_task_group() as tg:
        processes = [
            tg.soonify(asyncio.create_subprocess_exec)(
                "kafka-topics.sh",
                "--create",
                f"--topic={topic}",
                f"--bootstrap-server={bootstrap_server}",
                stdout=asyncio.subprocess.PIPE,
                stdin=asyncio.subprocess.PIPE,
            )
            for topic in self.topics
        ]

    try:
        return_values = [
            await asyncio.wait_for(process.value.wait(), 30) for process in processes
        ]
        if any(return_value != 0 for return_value in return_values):
            raise ValueError("Could not create missing topics!")
    except asyncio.TimeoutError as _:
        raise ValueError("Timed out while creating missing topics!")


@patch  # type: ignore
async def _start(self: LocalKafkaBroker) -> str:
    self._install()

    self.temporary_directory = TemporaryDirectory()
    self.temporary_directory_path = Path(self.temporary_directory.__enter__())

    await self._start_zookeeper()
    await self._start_kafka()

    listener_port = self.kafka_kwargs.get("listener_port", 9092)
    bootstrap_server = f"127.0.0.1:{listener_port}"
    logger.info(f"Local Kafka broker up and running on {bootstrap_server}")

    await self._create_topics()

    self._is_started = True

    return bootstrap_server


@patch  # type: ignore
async def _stop(self: LocalKafkaBroker) -> None:
    await terminate_asyncio_process(self.kafka_task)  # type: ignore
    await terminate_asyncio_process(self.zookeeper_task)  # type: ignore
    self.temporary_directory.__exit__(None, None, None)  # type: ignore
    self._is_started = False


In [ ]:
broker = LocalKafkaBroker(zookeeper_port=9799, listener_port=9689)
async with broker:
    pass

print("*" * 50 + "ZOOKEEPER LOGS" + "+" * 50)
zookeeper_output, _ = await broker.zookeeper_task.communicate()
print(zookeeper_output.decode("UTF-8"))

print("*" * 50 + "KAFKA LOGS" + "+" * 50)
kafka_output, _ = await broker.kafka_task.communicate()
print(kafka_output.decode("UTF-8"))

[INFO] __main__: Java is already installed.
[INFO] __main__: Kafka is already installed.
[INFO] __main__: Starting zookeeper...
[INFO] __main__: zookeeper started, sleeping for 5 seconds...
[INFO] __main__: Starting kafka...
[INFO] __main__: kafka started, sleeping for 5 seconds...
[INFO] __main__: Local Kafka broker up and running on 127.0.0.1:9689
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 164911...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 164911 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 164546...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 164546 terminated.
**************************************************ZOOKEEPER LOGS++++++++++++++++++++++++++++++++++++++++++++++++++
[2023-03-03 10:26:29,857] INFO Reading configuration from: /tmp/tmp80atlpl2/zookeeper.properties (org.apache.zookeeper.server.q

In [ ]:
port = 9939

broker_1 = LocalKafkaBroker(zookeeper_port=port, listener_port=9941)
broker_2 = LocalKafkaBroker(zookeeper_port=port, listener_port=9942)
async with broker_1:
    with pytest.raises(ValueError) as e:
        async with broker_2:
            pass

expected = (
    "Could not start zookeeper with params: {" + f"'zookeeper_port': {port}" + "}"
)
assert e.value.args == (expected,)

for broker in [broker_2]:
    print("*" * 50 + "ZOOKEEPER LOGS" + "+" * 50)
    zookeeper_output, _ = await broker.zookeeper_task.communicate()
    print(zookeeper_output.decode("UTF-8"))

In [ ]:
port = 9939

broker_1 = LocalKafkaBroker(zookeeper_port=port, listener_port=9941)
broker_2 = LocalKafkaBroker(zookeeper_port=port, listener_port=9941, retries=1)
async with broker_1:
    async with broker_2:
        pass

for broker in [broker_2]:
    print("*" * 50 + "ZOOKEEPER LOGS" + "+" * 50)
    zookeeper_output, _ = await broker.zookeeper_task.communicate()
    print(zookeeper_output.decode("UTF-8"))

In [ ]:
# | export


@patch  # type: ignore
def start(self: LocalKafkaBroker) -> str:
    """Starts a local kafka broker and zookeeper instance synchronously
    Returns:
       Kafka broker bootstrap server address in string format: add:port
    """
    logger.info(f"{self.__class__.__name__}.start(): entering...")
    try:
        # get or create loop
        try:
            loop = asyncio.get_event_loop()
        except RuntimeError as e:
            logger.warning(
                f"{self.__class__.__name__}.start(): RuntimeError raised when calling asyncio.get_event_loop(): {e}"
            )
            logger.warning(
                f"{self.__class__.__name__}.start(): asyncio.new_event_loop()"
            )
            loop = asyncio.new_event_loop()

        # start zookeeper and kafka broker in the loop

        if loop.is_running():
            if self.apply_nest_asyncio:
                logger.warning(
                    f"{self.__class__.__name__}.start(): ({loop}) is already running!"
                )
                logger.warning(
                    f"{self.__class__.__name__}.start(): calling nest_asyncio.apply()"
                )
                nest_asyncio.apply(loop)
            else:
                msg = f"{self.__class__.__name__}.start(): ({loop}) is already running! Use 'apply_nest_asyncio=True' when creating 'LocalKafkaBroker' to prevent this."
                logger.error(msg)
                raise RuntimeError(msg)

        try:
            retval = loop.run_until_complete(self._start())
            logger.info(f"{self.__class__}.start(): returning {retval}")
            return retval
        except RuntimeError as e:
            logger.warning(
                f"{self.__class__.__name__}.start(): RuntimeError raised for loop ({loop}): {e}"
            )
            logger.warning(
                f"{self.__class__.__name__}.start(): calling nest_asyncio.apply()"
            )
    finally:
        logger.info(f"{self.__class__.__name__}.start(): exited.")


@patch  # type: ignore
def stop(self: LocalKafkaBroker) -> None:
    """Stops a local kafka broker and zookeeper instance synchronously
    Returns:
       None
    """
    logger.info(f"{self.__class__.__name__}.stop(): entering...")
    try:
        if not self._is_started:
            raise RuntimeError(
                "LocalKafkaBroker not started yet, please call LocalKafkaBroker.start() before!"
            )

        loop = asyncio.get_event_loop()
        loop.run_until_complete(self._stop())
    finally:
        logger.info(f"{self.__class__.__name__}.stop(): exited.")

In [ ]:
broker = LocalKafkaBroker(
    zookeeper_port=9998, listener_port=9789, apply_nest_asyncio=True
)
with broker:
    print("Hello world!")

print("*" * 50 + "ZOOKEEPER LOGS" + "+" * 50)
zookeeper_output, _ = await broker.zookeeper_task.communicate()
print(zookeeper_output.decode("UTF-8"))


print("*" * 50 + "KAFKA LOGS" + "+" * 50)
kafka_output, _ = await broker.kafka_task.communicate()
print(kafka_output.decode("UTF-8"))

In [ ]:
with LocalKafkaBroker(
    zookeeper_port=9998, listener_port=9789, apply_nest_asyncio=True
) as bootstrap_servers:
    print(bootstrap_servers)
    assert bootstrap_servers == "127.0.0.1:9789"

    msgs = [
        dict(user_id=i, feature_1=[(i / 1_000) ** 2], feature_2=[i % 177])
        for i in trange(100_000, desc="generating messages")
    ]

    async with asyncer.create_task_group() as tg:
        tg.soonify(consumes_messages)(
            msgs_count=len(msgs), topic="test_data", bootstrap_servers=bootstrap_servers
        )

        await asyncio.sleep(1)

        tg.soonify(produce_messages)(
            msgs=msgs, topic="test_data", bootstrap_servers=bootstrap_servers
        )

In [ ]:
async with LocalKafkaBroker(
    zookeeper_port=9998, listener_port=9789
) as bootstrap_servers:
    print(bootstrap_servers)
    assert bootstrap_servers == "127.0.0.1:9789"

    msgs = [
        dict(user_id=i, feature_1=[(i / 1_000) ** 2], feature_2=[i % 177])
        for i in trange(100_000, desc="generating messages")
    ]

    async with asyncer.create_task_group() as tg:
        tg.soonify(consumes_messages)(
            msgs_count=len(msgs), topic="test_data", bootstrap_servers=bootstrap_servers
        )

        await asyncio.sleep(2)

        tg.soonify(produce_messages)(
            msgs=msgs, topic="test_data", bootstrap_servers=bootstrap_servers
        )

In [ ]:
topics = ["topic_1", "topic_2"]
async with LocalKafkaBroker(
    zookeeper_port=9788, listener_port=9579, topics=topics
) as bootstrap_server:
    task = await asyncio.create_subprocess_exec(
        "kafka-topics.sh",
        "--list",
        f"--bootstrap-server={bootstrap_server}",
        stdout=asyncio.subprocess.PIPE,
        stdin=asyncio.subprocess.PIPE,
    )
    output, _ = await asyncio.wait_for(task.communicate(), 5)
    listed_topics = output.decode("UTF-8").split("\n")[:-1]
    assert set(listed_topics) == set(topics)
print("ok")